# 09: Lambda Epressions
**autor: Erik Král ekral@utb.cz**

---


Obsah:
- Lambda výrazy
- Expression lambda
- Statement lambda



Lambda výrazy nám umoňují zapsat anonymní funkci, tedy funkci bez jména. Používám se s polečně s delegáty. V .NET 2.0 se pro stejný účel používali anonymní metody, tyto byly ale nahrazeny v .NET 3.0 lambda výrazy a už se nepoužívají.

### Expression Lambda

V následujícím příkladu přiřazujeme referenci na metodu `VratRetezec` delegátu typu `Func<int,bool, string>`.

In [ ]:
string VratRetezec(int x, bool y)
{
    return $"x ma hodnotu {x} a promena y ma hodnotu {y}";
}

Func<int, bool, string> delegat = VratRetezec;

string retezec = delegat.Invoke(2, true);

Console.WriteLine(retezec);

x ma hodnotu 2 a promena y ma hodnotu True


Pokud bychom tuto metodu používali volat jen jednou, nebo bychom chtěli zachytit lokální proměnné, tak ji nemusíme definovat jako metodu, ale můžeme použít lambda výraz. Všimněte si, že u parametrů `x` a `y` nemusíme uvádět typ a když metoda obsahuje pouze jeden příkaz, tak nepoužíváme ani klíčové slovo `return` a složené závorky. Zápis je potom velmi úsporný a takovému výrazu potom říkáme Expression Lambda.


In [2]:
Func<int,bool, string> delegat = (x, y) => $"x ma hodnotu {x} a promena y ma hodnotu {y}";

string retezec = delegat.Invoke(2, true);

Console.WriteLine(retezec);

x ma hodnotu 2 a promena y ma hodnotu True


Lambda výrazy se často používájí s technologi [LINQ](https://learn.microsoft.com/en-us/dotnet/csharp/programming-guide/concepts/linq/). Všimněte si, že pokud má lambda výraz jen jeden parametr tak můžeme vynechat i složené závorky. V následujícím příkladu získáme jen kladná čísla z listu.

In [4]:
List<int> cisla = new List<int> { -20, 2, 5, -2, 7, 8 };

List<int> kladna = cisla.Where(x => x > 0).OrderBy(x => x).ToList();

kladna

[ 2, 5, 7, 8 ]

Pojďme si vyzkoušet příklad se studentama.

In [ ]:
record Student (string Jmeno, double Prumer);

List<Student> studenti = [
    new Student("Jan", 1.5),
    new Student("Petr", 2.3),
    new Student("Eva", 1.8),
    new Student("Karel", 3.0),
    new Student("Anna", 5.0)
];

// Jmena studentu s prumerem vetsim nez 2.0, serazena podle jmena vzestupne
List<string> vybraniStudenti = studenti
    .Where(s => s.Prumer > 2.0)
    .Select(s => s.Jmeno)
    .OrderBy(j => j)
    .ToList();

vybraniStudenti                                        

[ Anna, Karel, Petr ]

V následujícím příkladu si všimněte, že v lambda výrazu používáme lokální proměnnou `min` a kopírujeme jen proměnné větší než tato hodnota. Říkáme že tato proměnná je **captured** a prodlouží se její **lifetime**. Pro lambda výraz se prakticky vegeneruje třída s fieldem pro lokální proměnnou. Ukázku jak vidí následující kód překladač jazyka C# najdete na stránkách [sharplab.io](https://sharplab.io/#v2:CYLg1APgAgTAjAWAFBQAwAJ1TgFgNzJpZwB0AMgJYB2AjgUoQMxYzoDC6yA3spplMyg50AWQAUASk5I+6HjNl9eizJQDOAFwA81DQD50AYwpqANgEN0AXnRUApgHd067boNd0AWhioANOhh/AFZ/b38Adn8ADnQAX3plFV10AFtqawCEhRUXHSp9dAA3Ow01CgzjM3MSAHUACzsAJzsxAA9rA3aDNKoJEgAVAHsXSXpZWORYoA==). 

In [16]:
List<int> cisla = new List<int> { -20, 2, 5, -2, 7, 8 };

int min = 2;

List<int> vetsi = cisla.Where(x => x > min).ToList();

vetsi

[ 5, 7, 8 ]

Lambda výraz je také možné prevést na strom výrazů, což se využívá v některých knihovnách například pro objektově relační mapování, kdy se ze stromu výrázů generuje výraz v jazyce SQL.

In [17]:
#r "nuget: Microsoft.EntityFrameworkCore.Sqlite"

Installed Packages Microsoft.EntityFrameworkCore.Sqlite, 9.0.10

In [21]:
using Microsoft.EntityFrameworkCore;

class Student
{
    public int Id { get; set; }
    public string Jmeno { get; set; }
    public double Prumer { get; set; }
}

class SkolaContext : DbContext
{
    public DbSet<Student> Studenti { get; set; }

    protected override void OnConfiguring(DbContextOptionsBuilder optionsBuilder)
    {
        optionsBuilder.UseSqlite("Data Source=skola.db");
    }
}
List<string> jmena = [];

using (var context = new SkolaContext())
{
    context.Database.EnsureDeleted();
    context.Database.EnsureCreated();

    context.Studenti.AddRange(
        new Student { Jmeno = "Jan", Prumer = 1.5 },
        new Student { Jmeno = "Petr", Prumer = 2.3 },
        new Student { Jmeno = "Eva", Prumer = 1.8 },
        new Student { Jmeno = "Karel", Prumer = 3.0 },
        new Student { Jmeno = "Anna", Prumer = 5.0 }
    );

    context.SaveChanges();

    jmena = context.Studenti
        .Where(s => s.Prumer > 2.0)
        .Select(s => s.Jmeno)
        .OrderBy(j => j)
        .ToList();
}

jmena


[ Anna, Karel, Petr ]

### Statements lambda

Pokud chceme v lambda výrazu použít více příkazu, tak musíme použít statements lambda. Statements lambda nejde převést na strom výrazů.

V následujícím příkazu opět filtrujeme jen kladná čísla z Listu, ale při každém testu, zda je číslo kladné, ještě zvyšíme hodnotu počítadla a vypíšeme ji na terminál, musíme proto využít statements labmda. Všimněte si, že jsme museli použít klíčové slovo `return` a složené závorky.

In [22]:
int pocitadlo = 0;

List<int> kladna = cisla.Where(x =>
{
    if(x > 0)
    {
        System.Console.WriteLine(++pocitadlo);
        return true;
    }

    return false;

}).ToList();

1
2
3
4


Více se o lambda výrazech můžete dozvědět například zde:

[Lambda expressions (C# reference). Microsoft Docs. 2022](https://docs.microsoft.com/en-us/dotnet/csharp/language-reference/operators/lambda-expressions)

---
## Cvičení

1. Úkol

Nahraďte metodu lambda výrazem.

In [25]:
int Soucet(int x, int y)
{
    return x + y;
}

Func<int, int, int> d1 = Soucet; // nahradit lambda vyrazem

int vysledek = d1.Invoke(2, 2);

Console.WriteLine(vysledek);

4


Rešení

In [23]:
Func<int, int, int> d1 = (x, y) => x + y; 

int vysledek = d1.Invoke(2, 2);

Console.WriteLine(vysledek);

4


2. Úkol

Nahraďte metodu lambda výrazem.

In [24]:
void Vypis(string text)
{
    Console.WriteLine(text);
}
       
Action<string> d2 = Vypis; // nahradit lambda vyrazem

d2.Invoke("Ahoj");

Ahoj


Řešení

In [27]:
Action<string> d2 = text => Console.WriteLine(text);

d2.Invoke("Ahoj"); 

Ahoj


3. Úkol

Nahraďte metodu lambda výrazem.

In [22]:
bool JeSude(int x)
{
    return x % 2 == 0;
}

Predicate<int> d3 = JeSude; // nahradit lambda vyrazem

bool vysledek3 = d3.Invoke(1);

Console.WriteLine(vysledek3);


False


Řešení

In [28]:
Predicate<int> d3 = x => x % 2 == 0;

bool vysledek3 = d3.Invoke(1);

Console.WriteLine(vysledek3);


False


4. Úkol

Nahraďte metodu lambda výrazem.

In [28]:
string PrevedNaText(int x)
{
    if(x > 0)
        return"kladne";
    else
        return "zaporne nebo 0";
}

Func<int, string> d4 = PrevedNaText;

string vysledek4 = d4.Invoke(1);

Console.WriteLine(vysledek4);

kladne


Řešení

In [29]:
Func<int, string> d4 = x => 
{
    if(x > 0)
        return"kladne";
    else
        return "zaporne nebo 0";
};

string vysledek4 = d4.Invoke(1);

Console.WriteLine(vysledek4);

kladne
